In [1]:
from aiida import orm, load_profile

load_profile('dev')

import numpy

from aiida import load_profile
load_profile() 
from aiida.orm import QueryBuilder, RemoteData, load_node
from aiida.orm import utils,Code, Computer, Data, Node, StructureData,Group, load_code, Str
from aiida.plugins import CalculationFactory,DataFactory
from aiida.orm.nodes.data.upf import get_pseudos_from_structure
from aiida.engine import submit

from aiida.plugins import WorkflowFactory

from aiida_quantumespresso_ph.workflows.dynamical_matrix import DynamicalMatrixWorkChain
from aiida_quantumespresso_ph.workflows.ph_interpolate import PhInterpolateWorkChain

/home/aiida/envs/aiida-super/code/aiida-core/aiida/plugins/entry_point.py:308: AiidaDeprecationWarning: The entry point `structure` is deprecated. Please replace it with `core.structure`.
  warn(
/home/aiida/envs/aiida-super/code/aiida-core/aiida/plugins/entry_point.py:308: AiidaDeprecationWarning: The entry point `singlefile` is deprecated. Please replace it with `core.singlefile`.
  warn(


Create a structure (5-atom cubic perovskite BaTiO3)

In [2]:
A_sites = ['Ba']
B_site = ['Ti']
a_arr=[3.977]
for a_site in A_sites:
    name = a_site[0]+B_site[0]+'O'

    #UNCOMMENT to create a group
    #slabel = 'structures/PhononTest'
    #gstruct = Group(label=slabel).store()
    

    for a in a_arr:
        StructureData = DataFactory("structure")
        alat =  a # angstrom
        cell = [[alat, 0., 0.,],
                [0., alat, 0.,],
                [0., 0., alat,]]
        s = StructureData(cell=cell)
        s.append_atom(position=(alat/2.,alat/2.,alat/2.), symbols=a_site)
        s.append_atom(position=(0.,0.,0.), symbols=B_site)
        s.append_atom(position=(alat/2.,0.,0.), symbols='O')
        s.append_atom(position=(0.,alat/2.,0.), symbols='O')
        s.append_atom(position=(0.,0.,alat/2.), symbols='O')
        
        #UNCOMMENT to store
        #snode=s.store()
        #gstruct.add_nodes([snode])

/home/aiida/envs/aiida-super/code/aiida-core/aiida/plugins/entry_point.py:308: AiidaDeprecationWarning: The entry point `structure` is deprecated. Please replace it with `core.structure`.
  warn(


ENTER CODES

In [5]:
codename_pw = #'pw-sirius-module@daint-s1073'
codename_ph = # 'ph-fromsiriusmod@daint-s1073'
codename_q2r = #'qe-6.4.1_q2r@theospc2'
codename_matdyn = #'qe-6.4.1_matdyn@theospc2'

<StructureData: uuid: ffc7a432-3fc4-480a-94d2-d78c26f31da5 (unstored)>

Create a group for the dynamcial matrix & interpolation calculations

In [ ]:
#clabel = 'calculations/PhononDynMatTest'
#ilabel = 'calculations/PhononInterpolationTest'
#gcalc = Group(label= clabel).store()
#gint = Group(label= ilabel).store()

Query if needed for structures

In [ ]:
#here just s

SUBMIT DynamicalMatrix workchain. Runs a pw followed by a PH
The are 'production' run parameters are commented out. The ones here are "good" and it will still run fast (loops over structures, but here there is just 1)

The qmesh of the PH run is the list of q-point the dynamical matrix will be calcuated at, theses are independent caluclations, so for testing you can choose [1,1,1] if you like

NEED TO ADD PSP family

In [ ]:
for [struct] in [s]: 
    Dict = DataFactory('dict')
    param_pw = Dict(dict={
            'CONTROL': {
                'calculation': 'scf',
                'restart_mode': 'from_scratch',
                'verbosity': 'high',
                'wf_collect': True,
                'tprnfor': True,
            },
            'SYSTEM': {
                'ecutwfc': 60, # 100,
                'ecutrho': 600, #1000,
                'occupations': 'fixed',
            },
            'ELECTRONS': {
                'conv_thr': 1.e-10, #1.e-12,
            }})
    
    KpointsData = DataFactory('array.kpoints')
    kpoints = KpointsData()
    kpoints.set_kpoints_mesh([8,8,8])

    param_ph = Dict(dict={
         'INPUTPH': {
                'tr2_ph' : 1.0e-16, # 1.0e-18,
                'epsil' : True,
                'recover': False,
    }})

    qpoints = KpointsData()
    qpoints.set_kpoints_mesh([2,2,2]) 
        
    inputs = {
            'pwWC': {
                'pw': {
                    'structure': struct,
                    'parameters':  param_pw,
                    'settings': Dict(dict={'cmdline': ["-nk", "2"]}),
                    'metadata': {
		        'options': {
                            'resources': {'num_machines': 2,'num_cores_per_mpiproc':1}
                            'max_wallclock_seconds': 5*60
                        },
                        'description': name+'scf dynmatWC'
                    },
                    'code': load_code(codename_pw)
                },
                'kpoints': kpoints,
	        'pseudo_family': Str('')#outdated I know 
            },
            'phWC': {
                'ph': {
                    'parameters':  param_ph,
                    #'settings': Dict(dict={'cmdline': ["-nk", "4"]}), #can be set but optimization is hard
                    'qpoints': qpoints,
                    'metadata': {
                        'options': {
                            'resources': {'num_machines': 2,'num_cores_per_mpiproc':1},
                            'max_wallclock_seconds': 3*60*60,
                            
                        },
                        'description': name+'ph dynmat'
                    },
                    'code': load_code(codename_ph)
                },
            },
            
        }
        
    result = submit(DynamicalMatrixWorkChain, **inputs)
    gcalc.add_nodes([result])
    

Collect results of Dynamical Matrix Workchain and run the necessary FT and interpolations to generate the phonon dispersion

In [ ]:
gs = Group.get(label = slabel)
gcalc = Group.get(label= clabel)
qb = QueryBuilder()
qb.append(Group, filters={'id': gs.pk}, tag='struct_group')
qb.append(StructureData, with_group='struct_group', tag='structs') 
qb.append(WorkChainNode,with_incoming='structs', tag='wfnodes',filters={'attributes.exit_status': {'==':0}}) #, project=['id'])
qb.append(Group, with_node = 'wfnodes',filters={'id': gcalc.pk}, tag='wfgroup')
qb.append(FolderData, with_incoming='wfnodes', tag='res', edge_filters={'label': 'ph_retrieved'}, project='id')
resWF = qb.all()

Calculation parameters for q2r and matdyn - gives the typical path through q-space, should be "connected" with seekpath

In [ ]:
KpointsData = DataFactory('array.kpoints')
qpoints = KpointsData()
qpoints.set_kpoints_mesh([2,2,2])


param_q2r = Dict(dict={
    'INPUT': {"zasr":"simple"}, #necessary as this is an insulator (LO-TO splitting), "crystal" would also work here
})

param_md = Dict(dict={
    'INPUT': {"asr":"simple"},
    })
d_qpoints = KpointsData()

n=30
qGX=[[0,0,i] for i in numpy.linspace(0,.5,n)]
qXM=[[0,i,.5] for i in numpy.linspace(0,.5,n)]
qMR=[[i,.5,.5] for i in numpy.linspace(0,.5,n)]
qRG=[[i,i,i] for i in numpy.linspace(.5,0,n)]
qGM=[[0,i,i] for i in numpy.linspace(0,.5,n)]
d_qpoints.set_kpoints(qGX[:-1]+qXM[:-1]+qMR[:-1]+qRG[:-1] + qGM)


SUBMIT interpolate work chain (really each run is ~10s so run locally if possible)

In [ ]:
for [n] in resWF:
    ret_folder = load_node(n)
    inputs = {
    'dynmat_folder': ret_folder,
    'q2r': {
        'q2r': {
            'parameters':  param_q2r,
            'metadata': {
                'options': {
                    'resources': {'num_machines': 1,'num_cores_per_mpiproc':1},
                    'max_wallclock_seconds': 2*60
                },
                'description': 'phono_disp_test q2r'
            },
            'code': load_code(codename_q2r),
        },
    },
    'matdyn': {
        'matdyn': {
            'parameters':  param_md,
            'kpoints': d_qpoints,
            'metadata': {
                'options': {
                    'resources': {'num_machines': 1,'num_cores_per_mpiproc':1},
                    'max_wallclock_seconds': 2*60
                },
                'description': 'phono_disp_test matdyrn'
            },
            'code': load_code(codename_matdyn),
        },
    },
    }

    result = submit(PhInterpolateWorkChain, **inputs)
    gint.add_nodes([result])